In [11]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import csv
import numpy as np
import os
from tqdm import tqdm

## Creating Small Subset of Data

### Import Data

In [2]:
rawDirectory = '\\\\isgs-sinkhole.ad.uillinois.edu\\geophysics\\Balikian\\BedrockWellData\\Wells\\RawWellData_OracleDatabase\\TxtData\\'
downholeDataFILE = "lcl_ISGS_DOWNHOLE_DATA.txt"
downholeDataDTYPES = {'ID':np.uint32, "API_NUMBER":np.uint64,"TABLE_NAME":str,"WHO":str,"INTERPRET_DATE":str,"FORMATION":str,"THICKNESS":np.float64,"TOP":np.float64,"BOTTOM":np.float64}
headerDataFILE = "lcl_ISGS_HEADER.txt"

In [3]:
downholeDataIN = pd.read_csv(rawDirectory+downholeDataFILE, sep=',', header='infer', encoding='latin-1')
downholeDataIN = downholeDataIN.dropna(subset=['API_NUMBER'])
downholeData = downholeDataIN.copy()

headerDataIN = pd.read_csv(rawDirectory+headerDataFILE, sep=',', header='infer', encoding='latin-1')
headerDataIN = headerDataIN.dropna(subset=['LATITUDE'])
headerDataIN = headerDataIN.dropna(subset=['LONGITUDE'])
headerData = headerDataIN[['API_NUMBER', 'LATITUDE', 'LONGITUDE']].copy()

C:\Users\jfranke3\AppData\Local\Temp\ipykernel_16828\2792783426.py:5: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  headerDataIN = pd.read_csv(rawDirectory+headerDataFILE, sep=',', header='infer', encoding='latin-1')


In [4]:
for i, row in downholeData.loc[downholeData['API_NUMBER'] == 480590038100].iterrows():
    print(i, row['API_NUMBER'])

3016936 480590038100.0
3016937 480590038100.0
3016938 480590038100.0
3016939 480590038100.0
3016940 480590038100.0
3016941 480590038100.0
3016942 480590038100.0


In [5]:
downholeData = pd.merge(downholeData, headerData, on='API_NUMBER', how='left')
downholeData.head(50)

downholeDataDTYPES = {'ID':np.uint32, "API_NUMBER":np.uint64,"TABLE_NAME":str,"WHO":str,"INTERPRET_DATE":str,"FORMATION":str,"THICKNESS":np.float64,"TOP":np.float64,"BOTTOM":np.float64, "LATITUDE":np.float64, "LONGITUDE":np.float64}
for i in range(0, np.shape(downholeData)[1]):
    downholeData.iloc[:,i] = downholeData.iloc[:,i].astype(downholeDataDTYPES[downholeData.iloc[:,i].name])
downholeData.tail(150)

,API_NUMBER,TABLE_NAME,WHO,INTERPRET_DATE,FORMATION,THICKNESS,TOP,BOTTOM,LATITUDE,LONGITUDE
3016935,480590037900,WFORMATIONS,nan,nan,sand & gravel,145.0,9.0,154.0,NaN,NaN
3016936,480590038100,WFORMATIONS,nan,nan,blue clay firm,18.0,32.0,50.0,NaN,NaN
3016937,480590038100,WFORMATIONS,nan,nan,clean fine gravel,150.0,5.0,155.0,NaN,NaN
3016938,480590038100,WFORMATIONS,nan,nan,gravely clay firm,50.0,10.0,60.0,NaN,NaN
3016939,480590038100,WFORMATIONS,nan,nan,hard brown clay,0.0,18.0,18.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
3017080,480650003600,FORMATION_TOPS,nan,nan,Platteville,NaN,217.0,NaN,NaN,NaN
3017081,480650003600,FORMATION_TOPS,nan,nan,Quimby Mill Dolomite,NaN,213.0,NaN,NaN,NaN
3017082,480650003600,FORMATION_TOPS,nan,nan,Spechts Ferry,NaN,211.0,NaN,NaN,NaN
3017083,480650003600,FORMATION_TOPS,nan,nan,St James,NaN,183.0,NaN,NaN,NaN


### Create subset of ~1000 random wells statewide

In [7]:
new_records_all_state = pd.DataFrame()
temp_api = 0
while new_records_all_state.shape[0] < 1000:
    temp_row = downholeData.sample()
    temp_api = temp_row['API_NUMBER']
    for i, row in downholeData.loc[downholeData['API_NUMBER'] == int(temp_api)].iterrows():
        new_records_all_state = new_records_all_state.append(row)
    new_records_all_state.drop_duplicates()


In [8]:
new_records_all_state

,API_NUMBER,TABLE_NAME,WHO,INTERPRET_DATE,FORMATION,THICKNESS,TOP,BOTTOM,LATITUDE,LONGITUDE
615671,120410068000,WFORMATIONS,nan,nan,"dol,wh,ptly vugy,chty",118.0,0.0,118.0,39.809712,-88.266917
615672,120410068000,WFORMATIONS,nan,nan,"dol,wh/gry/bf,vugy,O stn,vy slgtly sy",2.0,118.0,120.0,39.809712,-88.266917
615673,120410068000,WFORMATIONS,nan,nan,no samples(120-211'),91.0,120.0,211.0,39.809712,-88.266917
501258,120314827800,WFORMATIONS,nan,nan,clay,4.0,0.0,4.0,41.786732,-87.585270
501259,120314827800,WFORMATIONS,nan,nan,hard gray clay,5.0,64.0,69.0,41.786732,-87.585270
...,...,...,...,...,...,...,...,...,...,...
2110457,121452440200,FORMATION_TOPS,CO,nan,New Albany,NaN,3430.0,4383.0,37.958700,-89.268045
2110458,121452440200,FORMATION_TOPS,CO,nan,Osage,NaN,2815.0,NaN,37.958700,-89.268045
2110459,121452440200,FORMATION_TOPS,CO,nan,Salem,NaN,2285.0,NaN,37.958700,-89.268045
2110460,121452440200,FORMATION_TOPS,CO,nan,Ste Genevieve,NaN,1961.0,NaN,37.958700,-89.268045


## Creating Subset of Data based on location

In [9]:
for i in range(0, np.shape(downholeData)[1]):
    downholeData.iloc[:,i] = downholeData.iloc[:,i].astype(downholeDataDTYPES[downholeData.iloc[:,i].name])
downholeData

,API_NUMBER,TABLE_NAME,WHO,INTERPRET_DATE,FORMATION,THICKNESS,TOP,BOTTOM,LATITUDE,LONGITUDE
0,120010000200,FORMATION_TOPS,nan,nan,Cedar Valley,NaN,560.0,NaN,40.080721,-90.9211
1,120010000200,FORMATION_TOPS,nan,nan,Keokuk,NaN,175.0,NaN,40.080721,-90.9211
2,120010000200,FORMATION_TOPS,nan,nan,Kinderhook,NaN,380.0,NaN,40.080721,-90.9211
3,120010000200,FORMATION_TOPS,nan,nan,Pennsylvanian,NaN,5.0,NaN,40.080721,-90.9211
4,120010000200,FORMATION_TOPS,nan,nan,Silurian,NaN,576.0,NaN,40.080721,-90.9211
...,...,...,...,...,...,...,...,...,...,...
3017080,480650003600,FORMATION_TOPS,nan,nan,Platteville,NaN,217.0,NaN,NaN,NaN
3017081,480650003600,FORMATION_TOPS,nan,nan,Quimby Mill Dolomite,NaN,213.0,NaN,NaN,NaN
3017082,480650003600,FORMATION_TOPS,nan,nan,Spechts Ferry,NaN,211.0,NaN,NaN,NaN
3017083,480650003600,FORMATION_TOPS,nan,nan,St James,NaN,183.0,NaN,NaN,NaN


### Get all well data from one county

In [12]:
single_county_wells = pd.DataFrame()
location_code = "12019"
countyrange = downholeData.API_NUMBER.isin(range(120190000000, 120199999999))
for i, row in tqdm(downholeData.loc[countyrange].iterrows()):
    single_county_wells = single_county_wells.append(row)


for i in range(0, np.shape(single_county_wells)[1]):
    single_county_wells.iloc[:,i] = single_county_wells.iloc[:,i].astype(downholeDataDTYPES[single_county_wells.iloc[:,i].name])
single_county_wells = single_county_wells.reindex(columns=['API_NUMBER', 'TABLE_NAME', 'WHO', 'INTERPRET_DATE', 'FORMATION', 'THICKNESS', 'TOP', 'BOTTOM', 'LATITUDE', 'LONGITUDE'])
single_county_wells

52806it [12:34, 70.01it/s]


,API_NUMBER,TABLE_NAME,WHO,INTERPRET_DATE,FORMATION,THICKNESS,TOP,BOTTOM,LATITUDE,LONGITUDE
99334,120190002100,WFORMATIONS,nan,nan,blue clay - gravel showing and boulders,34.0,16.0,50.0,40.091445,-88.378863
99467,120190001100,WFORMATIONS,nan,nan,"sand, brownish gray, showing oil",2.0,280.0,282.0,39.907239,-88.068366
99468,120190001100,WFORMATIONS,nan,nan,"sand, fine, gray (no water)",10.0,232.0,242.0,39.907239,-88.068366
99469,120190001100,WFORMATIONS,nan,nan,"sand, fine, gray; water",2.0,277.0,279.0,39.907239,-88.068366
99470,120190001100,WFORMATIONS,nan,nan,"sand, gravel, and gray clay",30.0,20.0,50.0,39.907239,-88.068366
...,...,...,...,...,...,...,...,...,...,...
154177,120192810000,WFORMATIONS,nan,nan,sand,5.0,60.0,65.0,40.110472,-88.202139
154178,120192810100,WFORMATIONS,nan,nan,clay,24.0,0.0,24.0,40.110000,-88.230000
154179,120192810100,WFORMATIONS,nan,nan,glacial till,118.0,24.0,142.0,40.110000,-88.230000
154180,120192810100,WFORMATIONS,nan,nan,limestone,4.0,194.0,198.0,40.110000,-88.230000


### Get all wells from two specified counties

In [17]:
multi_county_wells = pd.DataFrame()
location_code = "12019"
countyrange = downholeData.API_NUMBER.isin(range(120010000000, 120019999999))
countyrange2 = downholeData.API_NUMBER.isin(range(121830000000, 121839999999))
for i, row in tqdm(downholeData.loc[countyrange].iterrows()):
    multi_county_wells = multi_county_wells.append(row)
for i, row in tqdm(downholeData.loc[countyrange2].iterrows()):
    multi_county_wells = multi_county_wells.append(row)

11736it [02:25, 80.83it/s]
37073it [09:04, 68.09it/s]


In [18]:
if os.path.exists("Well-Data-Subsets/state-wells.csv"):
    os.remove("Well-Data-Subsets/state-wells.csv")
if os.path.exists('Well-Data-Subsets/single_county_wells.csv'):
    os.remove('Well-Data-Subsets/single_county_wells.csv')
if os.path.exists('Well-Data-Subsets/multi_county_wells.csv'):
    os.remove('Well-Data-Subsets/multi_county_wells.csv')

new_records_all_state.to_csv('Well-Data-Subsets/state-wells.csv')
single_county_wells.to_csv('Well-Data-Subsets/single_county_wells.csv')
multi_county_wells.to_csv('Well-Data-Subsets/multi_county_wells.csv')

,API_NUMBER,TABLE_NAME,WHO,INTERPRET_DATE,FORMATION,THICKNESS,TOP,BOTTOM,LATITUDE,LONGITUDE
99334,120190002100,WFORMATIONS,nan,nan,blue clay - gravel showing and boulders,34.0,16.0,50.0,40.091445,-88.378863
99467,120190001100,WFORMATIONS,nan,nan,"sand, brownish gray, showing oil",2.0,280.0,282.0,39.907239,-88.068366
99468,120190001100,WFORMATIONS,nan,nan,"sand, fine, gray (no water)",10.0,232.0,242.0,39.907239,-88.068366
99469,120190001100,WFORMATIONS,nan,nan,"sand, fine, gray; water",2.0,277.0,279.0,39.907239,-88.068366
99470,120190001100,WFORMATIONS,nan,nan,"sand, gravel, and gray clay",30.0,20.0,50.0,39.907239,-88.068366
...,...,...,...,...,...,...,...,...,...,...
154177,120192810000,WFORMATIONS,nan,nan,sand,5.0,60.0,65.0,40.110472,-88.202139
154178,120192810100,WFORMATIONS,nan,nan,clay,24.0,0.0,24.0,40.110000,-88.230000
154179,120192810100,WFORMATIONS,nan,nan,glacial till,118.0,24.0,142.0,40.110000,-88.230000
154180,120192810100,WFORMATIONS,nan,nan,limestone,4.0,194.0,198.0,40.110000,-88.230000
